# Whale Optimisation Algorithms and its variations

In [7]:
import os
import opfunu
import numpy as np
import pandas as pd
from mealpy.optimizer import Optimizer

## Original Whale Optimisation Algorithm

In [5]:
class OriginalWOA(Optimizer):
    """
    The original version of: Whale Optimization Algorithm (WOA)

    Links:
        1. https://doi.org/10.1016/j.advengsoft.2016.01.008

    Examples
    ~~~~~~~~
    >>> import numpy as np
    >>> from mealpy.swarm_based.WOA import OriginalWOA
    >>>
    >>> def fitness_function(solution):
    >>>     return np.sum(solution**2)
    >>>
    >>> problem_dict1 = {
    >>>     "fit_func": fitness_function,
    >>>     "lb": [-10, -15, -4, -2, -8],
    >>>     "ub": [10, 15, 12, 8, 20],
    >>>     "minmax": "min",
    >>> }
    >>>
    >>> epoch = 1000
    >>> pop_size = 50
    >>> model = OriginalWOA(epoch, pop_size)
    >>> best_position, best_fitness = model.solve(problem_dict1)
    >>> print(f"Solution: {best_position}, Fitness: {best_fitness}")

    References
    ~~~~~~~~~~
    [1] Mirjalili, S. and Lewis, A., 2016. The whale optimization algorithm.
    Advances in engineering software, 95, pp.51-67.
    """

    def __init__(self, epoch=10000, pop_size=100, **kwargs):
        """
        Args:
            epoch (int): maximum number of iterations, default = 10000
            pop_size (int): number of population size, default = 100
        """
        super().__init__(**kwargs)
        self.name = "OriginalWOA"
        self.epoch = self.validator.check_int("epoch", epoch, [1, 100000])
        self.pop_size = self.validator.check_int("pop_size", pop_size, [10, 10000])
        self.set_parameters(["epoch", "pop_size"])
        self.sort_flag = False

    def evolve(self, epoch):
        """
        The main operations (equations) of algorithm. Inherit from Optimizer class

        Args:
            epoch (int): The current iteration
        """
        a = 2 - 2 * epoch / (self.epoch - 1)  # linearly decreased from 2 to 0
        pop_new = []
        for idx in range(0, self.pop_size):
            r = np.random.rand()
            A = 2 * a * r - a
            C = 2 * r
            l = np.random.uniform(-1, 1)
            p = 0.5
            b = 1
            if np.random.uniform() < p:
                if np.abs(A) < 1:
                    D = np.abs(
                        C * self.g_best[self.ID_POS] - self.pop[idx][self.ID_POS]
                    )
                    pos_new = self.g_best[self.ID_POS] - A * D
                else:
                    # x_rand = pop[np.random.np.random.randint(self.pop_size)]         # select random 1 position in pop
                    x_rand = self.create_solution(self.problem.lb, self.problem.ub)
                    D = np.abs(C * x_rand[self.ID_POS] - self.pop[idx][self.ID_POS])
                    pos_new = x_rand[self.ID_POS] - A * D
            else:
                D1 = np.abs(self.g_best[self.ID_POS] - self.pop[idx][self.ID_POS])
                pos_new = (
                    self.g_best[self.ID_POS]
                    + np.exp(b * l) * np.cos(2 * np.pi * l) * D1
                )
            pos_new = self.amend_position(pos_new, self.problem.lb, self.problem.ub)
            pop_new.append([pos_new, None])
            if self.mode not in self.AVAILABLE_MODES:
                target = self.get_target_wrapper(pos_new)
                self.pop[idx] = self.get_better_solution(
                    self.pop[idx], [pos_new, target]
                )
        if self.mode in self.AVAILABLE_MODES:
            pop_new = self.update_target_wrapper_population(pop_new)
            print(len(pop_new))
            self.pop = self.greedy_selection_population(self.pop, pop_new)

# Results Generation

In [5]:
benchmark_dict = {
  2014: range(1, 31),
  2017: range(1, 30),
}

In [53]:
def final_result(model_name):
  WOAResults = {}
  for year in [2014, 2017]:
    for func_num in benchmark_dict[year]:
      func_map_key_name = f'F{func_num}{year}'
      WOAResults[func_map_key_name] = {}
      for dimension in [10, 30, 50, 100]:
        func_name = f'opfunu.cec_based.F42014(ndim={dimension})'
        WOAResults[func_map_key_name][dimension] = {}
        problem = eval(func_name)
        problem_dict = {
          "fit_func": problem.evaluate,
          "lb": problem.lb,
          "ub": problem.ub,
          "minmax": "min",
          "log_to": "file",
          "log_file": "history.log",
          "dim" : dimension
        }
        for pop_size in [10, 20, 30, 40, 50, 70, 100, 200, 300, 400, 500, 700, 1000]:
          run_result = np.array([])
          for _ in range(1, 52):
            epoch = 10000
            pop_size = pop_size
            model = model_name(epoch, pop_size)
            max_fe = 10000 * dimension
            term_dict = {
              "max_fe": max_fe
            }
            best_position, best_fitness = model.solve(problem_dict, termination=term_dict)
            run_result = np.append(run_result, best_fitness)
          WOAResults[func_map_key_name][dimension][pop_size] = np.mean(run_result)
  for key in WOAResults.keys():
    df = pd.DataFrame(WOAResults[key])
    directory_path = f'./results/{model.name}'
    if not os.path.exists(directory_path):
      os.makedirs(directory_path)
    df.to_csv(f'{directory_path}/{key}.csv', index = True)

In [88]:
class ImprovedWhaleOptimization():
    """class implements the whale optimization algorithm as found at
    http://www.alimirjalili.com/WOA.html
    and
    https://doi.org/10.1016/j.advengsoft.2016.01.008
    """
    """initialize solutions uniform randomly in space"""

    def __init__(self, opt_func, constraints, nsols, b, a, a_step, ndim, maximize=False):
        self._fe = 0
        self._ndim = ndim
        self._opt_func = opt_func # fitness function -> solve method
        self._constraints = constraints # lb, ub -> solve
        self._b = b
        self._a = a
        self._a_step = a_step
        self._maximize = maximize
        self._nsols = nsols
        self._best_solutions = []
        self._sols = self._init_solutions(nsols)

    def get_solutions(self):
        """return solutions"""
        return self._sols

    def optimize(self, t, ngens, max_fe):
        """solutions randomly encircle, search or attack"""
        ranked_sol = self._rank_solutions(self._sols)
        best_sol = ranked_sol[0]
        #include best solution in next generation solutions
        new_sols = [best_sol]

        for s in ranked_sol[1:]:
            if np.random.uniform(0.0, 1.0) > 0.5:
                A = self._compute_A()
                norm_A = np.linalg.norm(A)
                if norm_A < 1.0:
                    new_s = self._encircle(s, best_sol, A)
                else:
                    ###select random sol
                    random_sol = self._sols[np.random.randint(self._sols.shape[0])]
                    new_s = self._search(s, random_sol, A)
            else:
                new_s = self._attack(s, best_sol)
            new_sols.append(self._constrain_solution(new_s))

        de_sol = []
        for s in new_sols:

            if self._fe >= max_fe:
                return

            # mutation
            indices = np.random.randint(0, self._sols.shape[0], (3))

            r1 = self._sols[indices[0]]
            r2 = self._sols[indices[1]]
            r3 = self._sols[indices[2]]

            f = np.random.uniform(0,1)
            v = r1 + f*(r2-r3)
            jrand = np.random.randint(self._sols.shape[1])

            u = v
            #crossover
            for j in range(len(s)):
              if j == jrand:
                u[j] = v[j]
              else:
                u[j] = s[j]

            #selection
            self._fe+=1
            fitness_s = self._opt_func(s)
            self._fe+=1
            fitness_u = self._opt_func(u)
            if self._maximize:
                if fitness_u > fitness_s:
                    de_sol.append(u)
                else:
                    de_sol.append(s)
            else:
                if fitness_u < fitness_s:
                    de_sol.append(u)
                else:
                    de_sol.append(s)

        self._sols = np.stack(de_sol)
        self._a -= self._a_step

    def EOBL(self, sol):
        
        ranked_sol = self._rank_solutions(sol)
        elite_sol = ranked_sol[0]
        opposition_sols = []
        lb = np.min(ranked_sol, axis=0)
        ub = np.max(ranked_sol, axis=0)

        for s in ranked_sol[0:]:
            k = np.random.uniform(0.0, 1.0, size=self._ndim)
            opp_sol = k*(lb+ub)-elite_sol
            constrain_opp_s = []
            for c, s in zip(self._constraints, opp_sol):
                if c[0] > s or c[1] < s:
                    s = np.random.uniform(lb, ub, size=self._ndim)
            constrain_opp_s.append(s)
            opposition_sols.append(constrain_opp_s)

        opposition_sols = np.stack(opposition_sols)

        return opposition_sols

    def _init_solutions(self, nsols):
        """initialize solutions uniform randomly in space"""
        sols = []
        sols = np.random.uniform(self._constraints[0], self._constraints[1], size=(nsols, self._ndim))
        # print(sols)
        sols = np.stack(sols, axis=-1)
        opp_sols = self.EOBL(sols)

        sols = np.concatenate((sols, opp_sols[0]))
        fitness = []
        for s in sols:
            self._fe+=1
            fitness.append(self._opt_func(s))
        sol_fitness = [(f, s) for f, s in zip(fitness, sols)]
        ranked_sol = list(sorted(sol_fitness, key=lambda x:x[0], reverse=self._maximize))
        eobl_sol = []

        for i in range(nsols):
          eobl_sol.append(ranked_sol[i][1])

        sols = np.stack(eobl_sol, axis=0)

        return sols

    def _constrain_solution(self, sol):
        """ensure solutions are valid wrt to constraints"""
        constrain_s = []
        
        # print(self._constraints)

        for i in range(self._ndim):
            if sol[i] < self._constraints[0][i]:
                sol[i] = self._constraints[0][i]
            if sol[i] > self._constraints[1][i]:
                sol[i] = self._constraints[1][i]
            
            constrain_s.append(sol[i])

        return constrain_s

        # for s in sol:
        #     print(s)
        #     if s < self._constraints[0]:
        #         s = self._constraints[0]
        #     if s > self._constraints[1]:
        #         s = self._constraints[1]
                
            # constrain_s.append(s)
        # return constrain_s

    def _rank_solutions(self, sol):
        """find best solution"""
        fitness = []
        for s in sol:
            self._fe+=1
            fitness.append(self._opt_func(s))
        sol_fitness = [(f, s) for f, s in zip(fitness, sol)]
        #best solution is at the front of the list
        ranked_sol = list(sorted(sol_fitness, key=lambda x:x[0], reverse=self._maximize))
        self._best_solutions.append(ranked_sol[0])

        return [ s[1] for s in ranked_sol]

    def best_solutions(self):
        # print('generation best solution history')
        # print('([fitness], [solution])')
        # for s in self._best_solutions:
        #     print(s)
        # print('\n')
        # print('best solution')
        # print('([fitness], [solution])')
        return sorted(self._best_solutions, key=lambda x:x[0], reverse=self._maximize)[0]

    def _compute_A(self):
        r = np.random.uniform(0.0, 1.0, size=self._ndim)
        return (2.0*np.multiply(self._a, r))-self._a

    def _compute_C(self):
        return 2.0*np.random.uniform(0.0, 1.0, size=self._ndim)

    def _encircle(self, sol, best_sol, A):
        D = self._encircle_D(sol, best_sol)
        return best_sol - np.multiply(A, D)

    def _encircle_D(self, sol, best_sol):
        C = self._compute_C()
        D = np.linalg.norm(np.multiply(C, best_sol)  - sol)
        return D

    def _search(self, sol, rand_sol, A):
        D = self._search_D(sol, rand_sol)
        return rand_sol - np.multiply(A, D)

    def _search_D(self, sol, rand_sol):
        C = self._compute_C()
        return np.linalg.norm(np.multiply(C, rand_sol) - sol)

    def _attack(self, sol, best_sol):
        D = np.linalg.norm(best_sol - sol)
        L = np.random.uniform(-1.0, 1.0, size=self._ndim)
        return np.multiply(np.multiply(D,np.exp(self._b*L)), np.cos(2.0*np.pi*L))+best_sol

In [89]:
class IWOA():

  def __init__(self, epoch, pop_size):
    self.epoch = epoch
    self.pop_size = pop_size

  def solve(self, problem_dict, termination):

      opt_func = problem_dict["fit_func"]

      b = 0.5
      a = 2.0
      a_step = 0.06
      constraints = [problem_dict["lb"], problem_dict["ub"]]
      if problem_dict["minmax"] == "min":
        maximize = False
      else:
        maximize = True

      opt_alg = ImprovedWhaleOptimization(opt_func, constraints, self.pop_size, b, a, a_step, problem_dict["dim"], maximize)

      for t in range(self.epoch):
          opt_alg.optimize(t, self.epoch, termination["max_fe"])
      return opt_alg.best_solutions()

In [92]:
final_result(IWOA)

KeyboardInterrupt: 

## Original Whale Optimisation Algorithm

In [18]:
final_result(OriginalWOA)

2023/10/04 06:23:00 PM, INFO, __main__.OriginalWOA: Solving single objective optimization problem.
2023/10/04 06:23:00 PM, INFO, __main__.OriginalWOA: >Problem: P, Epoch: 1, Current best: 1608.3068268294792, Global best: 1608.3068268294792, Runtime: 0.00680 seconds
2023/10/04 06:23:00 PM, INFO, __main__.OriginalWOA: >Problem: P, Epoch: 2, Current best: 1600.7870088917118, Global best: 1600.7870088917118, Runtime: 0.00503 seconds
2023/10/04 06:23:00 PM, INFO, __main__.OriginalWOA: >Problem: P, Epoch: 3, Current best: 1417.7444247500705, Global best: 1417.7444247500705, Runtime: 0.00397 seconds
2023/10/04 06:23:00 PM, INFO, __main__.OriginalWOA: >Problem: P, Epoch: 4, Current best: 1292.476715618967, Global best: 1292.476715618967, Runtime: 0.00488 seconds
2023/10/04 06:23:00 PM, INFO, __main__.OriginalWOA: >Problem: P, Epoch: 5, Current best: 1229.7340211166313, Global best: 1229.7340211166313, Runtime: 0.00220 seconds
2023/10/04 06:23:00 PM, INFO, __main__.OriginalWOA: >Problem: P, Epo

2023/10/04 06:23:01 PM, INFO, __main__.OriginalWOA: >Problem: P, Epoch: 18, Current best: 821.5062901500656, Global best: 821.5062901500656, Runtime: 0.00463 seconds
2023/10/04 06:23:01 PM, INFO, __main__.OriginalWOA: >Problem: P, Epoch: 19, Current best: 818.6496995403879, Global best: 818.6496995403879, Runtime: 0.00439 seconds
2023/10/04 06:23:01 PM, INFO, __main__.OriginalWOA: >Problem: P, Epoch: 20, Current best: 814.790798408892, Global best: 814.790798408892, Runtime: 0.00220 seconds
2023/10/04 06:23:01 PM, INFO, __main__.OriginalWOA: >Problem: P, Epoch: 21, Current best: 810.9260312751326, Global best: 810.9260312751326, Runtime: 0.00813 seconds
2023/10/04 06:23:01 PM, INFO, __main__.OriginalWOA: >Problem: P, Epoch: 22, Current best: 807.2910991047615, Global best: 807.2910991047615, Runtime: 0.00229 seconds
2023/10/04 06:23:01 PM, INFO, __main__.OriginalWOA: >Problem: P, Epoch: 23, Current best: 807.0213004882455, Global best: 807.0213004882455, Runtime: 0.00455 seconds
2023/1

KeyboardInterrupt: 

In [16]:
WOAResults

NameError: name 'WOAResults' is not defined

# Population Analysis

In [ ]:
import matplotlib.pyplot as plt
def plot_dimension(model_names, year, dimension, title="Dimension CEC", file_name_queries=None, special=None):
  for model in model_names:
    folder_path = f'./results/{model}'
    all_dataframes = []
    file_list = os.listdir(folder_path)
    for file_name in file_list:
      if file_name_queries is None:
        if file_name.endswith(f'{year}.csv'):
          file_path = os.path.join(folder_path, file_name)
          print(file_path)
          df = pd.read_csv(file_path)
          df = df.transpose()
          all_dataframes.append(df)
      else:
        if file_name in file_name_queries and file_name.endswith(f'{year}.csv'):
          file_path = os.path.join(folder_path, file_name)
          print(file_path)
          df = pd.read_csv(file_path)
          df = df.transpose()
          all_dataframes.append(df)
    combined_df = pd.concat(all_dataframes, axis=0)
    average_df = combined_df.groupby(level=0).mean()
    average_df = average_df.transpose()
    average_df['Rank'] = average_df['10'].rank()
    plt.plot(average_df['Unnamed: 0'].astype(int).astype(str), average_df['Rank'], marker='o', linewidth=1, label=model)
    plt.title(f'{dimension} {title} {year}')
    plt.xlabel('Population Size')
    plt.ylabel('Average Rank')
    plt.grid(True, axis="y")
    plt.legend()
    if special is None:
      saving_file_name = f'./results/Plots/{dimension}-CEC-{year}.png'
    else:
      saving_file_name = f'./results/Plots/{dimension}-{special}-CEC-{year}.png'
    plt.savefig(saving_file_name, dpi=300, bbox_inches='tight')
  plt.clf()

In [ ]:
all_algos = ['OriginalWOA', 'HI_WOA']

# Overall Average

In [ ]:
for year in [2014, 2017]:
  for dimension in [10, 30, 50, 100]:
    plot_dimension(all_algos, year, dimension)

# Functions Categorization

In [ ]:
unimodal_functions = []
composition_functions = []
typical_multimodal_functions = []

for func_num in range(1, 4):
  unimodal_functions.append(f'F{func_num}2014.csv')
for func_num in range(1, 3):
  unimodal_functions.append(f'F{func_num}2017.csv')

for func_num in range(23, 31):
  composition_functions.append(f'F{func_num}2014.csv')
for func_num in range(20, 30):
  composition_functions.append(f'F{func_num}2017.csv')

for func_num in range(4, 22):
  typical_multimodal_functions.append(f'F{func_num}2014.csv')
for func_num in range(3, 20):
  typical_multimodal_functions.append(f'F{func_num}2017.csv')
    

## Unimodal Functions

In [ ]:
for year in [2014, 2017]:
  for dimension in [10, 30, 50, 100]:
    plot_dimension(all_algos, year, dimension, "Dimension Unimodal", unimodal_functions, "unimodal")

## Composition Functions

In [ ]:
for year in [2014, 2017]:
  for dimension in [10, 30, 50, 100]:
    plot_dimension(all_algos, year, dimension, "Dimension Composition", composition_functions, "composition")